# 평균

- 주어진 동영상 파일에서 화면 전환 시간을 탐지하여 해당 시간을 출력
- 동영상의 프레임을 읽어와 이전 프레임과의 차이를 계산하고, 이를 평균값과 비교하여 임계값을 초과하는 경우 화면 전환으로 간주.
- 감지된 전환 타임프레임을 기반으로 FPS 값을 고려하여 전환 시간을 계산하고 출력

In [12]:
# 평균

import cv2

# 동영상 파일 경로
video_path = "/workspace/scene_change/kimbap.mp4"

# 동영상 캡쳐 객체 생성
cap = cv2.VideoCapture(video_path)

# 이전 프레임
prev_frame = None

# 화면 전환 타임프레임 리스트
transitions = []

# FPS 구하기
fps = cap.get(cv2.CAP_PROP_FPS)

threshold = 40

while True:
    # 프레임 읽기
    ret, frame = cap.read()

    # 프레임을 읽을 수 없으면 종료
    if not ret:
        break

    # 현재 프레임과 이전 프레임이 모두 있을 때
    if prev_frame is not None:
        # 현재 프레임과 이전 프레임의 차이 계산
        diff = cv2.absdiff(frame, prev_frame)

        # 차이의 평균 계산
        diff_mean = diff.mean()

        # 차이의 평균이 일정 값 이상이면 화면 전환으로 간주
        if diff_mean > threshold:
            # 화면 전환 타임프레임 추가
            transitions.append(cap.get(cv2.CAP_PROP_POS_FRAMES))

    # 현재 프레임을 이전 프레임으로 저장
    prev_frame = frame

# 동영상 캡쳐 객체 해제
cap.release()

# 전환 시간 계산
transition_times = [(frame / fps) for frame in transitions]

# 화면 전환 시간 출력
print("화면 전환 시간: ", transition_times)


화면 전환 시간:  [4.7047, 8.041366666666667, 10.477133333333333, 15.5155, 16.7167, 18.551866666666665, 19.886533333333333, 23.823800000000002, 26.559866666666668, 29.162466666666667, 31.5315, 36.20283333333333, 45.11173333333333, 48.38166666666667, 51.618233333333336, 53.653600000000004, 83.28320000000001, 87.02026666666667, 90.8908, 94.32756666666667, 104.57113333333334, 113.113, 116.7166, 124.45766666666667, 127.3272, 139.07226666666668, 142.10863333333333, 144.4443, 149.81633333333335, 154.32083333333333, 157.69086666666666, 165.8657, 176.80996666666667, 184.41756666666666, 193.82696666666666, 195.72886666666668, 198.16463333333334, 205.03816666666668, 216.44956666666667, 226.29273333333333, 232.56566666666666, 237.47056666666668, 242.17526666666666]


# 합

In [ ]:
# 합

import cv2

# 동영상 파일 경로
video_path = "/workspace/scene_change/IMG_4247.mp4"

# 동영상 캡쳐 객체 생성
cap = cv2.VideoCapture(video_path)

# 이전 프레임
prev_frame = None

# 화면 전환 타임프레임 리스트
transitions = []

# FPS 구하기
fps = cap.get(cv2.CAP_PROP_FPS)

while True:
    # 프레임 읽기
    ret, frame = cap.read()

    # 프레임을 읽을 수 없으면 종료
    if not ret:
        break

    # 현재 프레임과 이전 프레임이 모두 있을 때
    if prev_frame is not None:
        # 현재 프레임과 이전 프레임의 차이 계산
        diff = cv2.absdiff(frame, prev_frame)

        # 차이의 합계 계산
        diff_sum = diff.sum()

        # 차이의 합계가 일정 값 이상이면 화면 전환으로 간주
        if diff_sum > 50000000:
            # 화면 전환 타임프레임 추가
            transitions.append(cap.get(cv2.CAP_PROP_POS_FRAMES))

    # 현재 프레임을 이전 프레임으로 저장
    prev_frame = frame

# 동영상 캡쳐 객체 해제
cap.release()

# 전환 시간 계산
transition_times = [(frame / fps) for frame in transitions]

# 화면 전환 시간 출력
print("화면 전환 시간: ", transition_times)
